Data Preprocessing || `Model Training` || Streamlit App

---
# Solar Up -- Solar Panel Object Detection and Segmentation

WBS Coding School || Data Science Bootcamp || Final Project

---
<br>

**Use Google's GPU** to run this script to substantially reduce training time.

(Next to "Connection" -> Click arrow -> "Change runtime type" -> choose a GPU, e.g. "T4 GPU")


### Table of Contents
- [1. Download Data](#download)
- [2. Model Training](#training)
- [3. Export Model Weights](#export)

#### Libraries



In [ ]:
# ultralytics is the package you need for YOLOv8.
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# Update gdown for full functionality.
%pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [ ]:
import os
import shutil
import torch

import ultralytics
from ultralytics import YOLO
from ultralytics import settings

from google.colab import files, drive

___
<a id="download"></a>
## 1.&nbsp; Download Data

We'll download the satellite images and a config yaml file from a Google Drive data dump.

The yaml tells the YOLOv8 model where to find the data and what the classes are. In our case, there's only one class, solar panels.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download kasmi_solar_detection.yaml config file.
!gdown 1niMiTCjQN1FGxanL-hK8XBU4vgKB5FfT

# Insert the respective file ID here. To get it,
# go on your Google Drive > right click on 'kasmi_solar_detection.yaml' > Share > Copy link.
# The link will look something like:
# https://drive.google.com/file/d/<your_file_ID_here>/

Downloading...
From: https://drive.google.com/uc?id=1niMiTCjQN1FGxanL-hK8XBU4vgKB5FfT
To: /content/kasmi_solar_detection.yaml
100% 215/215 [00:00<00:00, 1.24MB/s]


Next, download the satellite images from a data-dump Drive.

In [ ]:
# Download zipped training data folder.
!gdown 1qMdHqmdVZynqZbgoB9dz1Ni6ZbFtaKCJ

Downloading...
From (original): https://drive.google.com/uc?id=1qMdHqmdVZynqZbgoB9dz1Ni6ZbFtaKCJ
From (redirected): https://drive.google.com/uc?id=1qMdHqmdVZynqZbgoB9dz1Ni6ZbFtaKCJ&confirm=t&uuid=14490323-57ca-48b0-bf5d-49128db4ffe2
To: /content/kasmi_solar_detection.zip
100% 1.57G/1.57G [00:11<00:00, 141MB/s]


In [ ]:
# Display current path on Google Colab Machine.
%pwd
# Current path is '/content'.

# YOLOv8 expects the data folder to be contained in a directory called 'content/datasets/'.
!mkdir /content/datasets/


mkdir: cannot create directory ‘/content/datasets/’: File exists
Archive:  kasmi_solar_detection.zip
checkdir:  cannot create extraction directory: content/datasets/kasmi_solar_detection
           No such file or directory


In [ ]:
# Unzip training data to that directory.
!unzip kasmi_solar_detection.zip -d /content/datasets/kasmi_solar_detection

'/content'

Check hardware and directory structure. Make sure a GPU is being used for model training.

In [ ]:
ultralytics.checks()

Ultralytics YOLOv8.1.28 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.5/78.2 GB disk)


In [ ]:
# Name of the dataset.
dataset_name = "kasmi_solar"

___
<a id="training"></a>
## 2.&nbsp; Model Training

Here we'll load a pretrained YOLOv8 model (small, medium or large) and train it on the solar panel satellite data. Subsequently, we'll check its performance on the validation set.

In [ ]:
# Load a COCO-pretrained YOLOv8m model.
model_name = 'yolov8m'
model = YOLO(f'{model_name}.pt')

100%|██████████| 49.7M/49.7M [00:00<00:00, 154MB/s]


In [ ]:
# Train the model on our custom dataset.
config_file = f'{dataset_name}_detection.yaml'
epochs = 20
img_size = 400
experiment = f'{model_name}_{epochs}'
device = 0 if torch.cuda.is_available() else "cpu" # device agnostic code

results = model.train(data=config_file, epochs=epochs, imgsz=img_size, name=experiment, device=device)

Ultralytics YOLOv8.1.28 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=kasmi_solar_detection.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=400, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov8m_20, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning /content/datasets/kasmi_solar_detection/train/labels.cache... 9312 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9312/9312 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/datasets/kasmi_solar_detection/val/labels.cache... 1996 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1996/1996 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8m_20/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 2 dataloader workers
Logging results to runs/detect/yolov8m_20
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      19/20      3.95G     0.5538     0.3244     0.8948         19        416: 100%|██████████| 582/582 [02:32<00:00,  3.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:17<00:00,  3.67it/s]

                   all       1996       2581      0.943      0.939       0.98      0.861



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      3.94G     0.5526     0.3172     0.8997         20        416:  58%|█████▊    | 337/582 [01:28<01:01,  3.97it/s]

In [ ]:
# Validate the model.
metrics = model.val(data=config_file, epochs=epochs, imgsz=img_size, name=experiment, device=device)  # no arguments needed, dataset and settings remembered

metrics.box.map    # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps   # a list contains map50-95(B) of each category

___
<a id="export"></a>
## 3.&nbsp; Export Model Weights

In [ ]:
# Load the custom trained model.
path_to_best = f"runs/detect/{experiment}/weights/best.pt" # .pt = PyTorch model
model = YOLO(path_to_best)

# Export the model. The default format is 'torchscript' (PyTorch).
model.export()

In [ ]:
# Specify the output path for the ZIP archive for the training output.
output_path = f"models/trained_model_{experiment}"
print(output_path)

shutil.make_archive(output_path, 'zip', f"/content/runs/detect/{experiment}")

In [ ]:
# Download trained model.
files.download(f"runs/detect/{experiment}/weights/best_{experiment}.torchscript")

In [ ]:
# Download training data.
files.download(f"models/training_data_{experiment}.zip")

In [ ]:
# Specify the output path for the ZIP archive for the evaluation data.
output_path = f"models/evaluation_data_{experiment}"
print(output_path)

shutil.make_archive(output_path, 'zip', f"/content/runs/detect/{experiment}")

In [ ]:
# Download evaluation data.
files.download(f"models/evaluation_data_{experiment}.zip")